In [1]:
%matplotlib inline

In [2]:
import os

In [3]:
%run notebook_setup

# Load data in from Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Transit fitting

*exoplanet* includes methods for computing the light curves transiting planets.
In its simplest form this can be used to evaluate a light curve like you would do with [batman](https://astro.uchicago.edu/~kreidberg/batman/), for example:

In [4]:
import os
HOME = os.environ['HOME']
os.chdir(f'{HOME}/Research/Planets/WASP43/github_analysis/notebooks')

In [5]:
import exoplanet as xo
import joblib
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import pandas as pd
import pymc3 as pm

from exomast_api import exoMAST_API
from statsmodels.robust.scale import mad
from time import time
# from tqdm import tqdm_notebook as tqdm

from arctor import create_raw_lc_stddev, Arctor, run_multiple_pymc3#, setup_and_plot_GTC
from arctor.utils import setup_and_plot_GTC
from arctor.utils import fit_2D_time_vs_other

In [6]:
def debug_message(message, end='\n'):
    print(f'[DEBUG] {message}', end=end)


def warning_message(message, end='\n'):
    print(f'[WARNING] {message}', end=end)


def info_message(message, end='\n'):
    print(f'[INFO] {message}', end=end)

In [7]:
plot_verbose = False
save_now = False
planet_name = 'WASP43'
file_type = 'flt.fits'

HOME = os.environ['HOME']
base_dir = os.path.join(HOME, 'Research', 'Planets', 'WASP43')
data_dir = os.path.join(base_dir, 'data', 'UVIS', 'MAST_2019-07-03T0738')
data_dir = os.path.join(data_dir, 'HST', 'FLTs')
working_dir = os.path.join(base_dir, 'github_analysis', 'savefiles')

In [8]:
planet = Arctor(planet_name, data_dir, working_dir, file_type)
joblib_filename = 'WASP43_savedict_206ppm_100x100_finescale.joblib.save'
joblib_filename = f'{working_dir}/{joblib_filename}'
planet.load_dict(joblib_filename)

[INFO] Initializing Instance of the `Arctor` Object


In [9]:
planet.photometry_df.columns

Index(['aperture_sum_11x41', 'aperture_sum_12x41', 'aperture_sum_13x41',
       'aperture_sum_14x41', 'aperture_sum_15x41', 'aperture_sum_16x41',
       'aperture_sum_17x41', 'aperture_sum_18x41', 'aperture_sum_19x41',
       'aperture_sum_20x41',
       ...
       'aperture_sum_23x60', 'aperture_sum_24x60', 'aperture_sum_25x60',
       'aperture_sum_26x60', 'aperture_sum_27x60', 'aperture_sum_28x60',
       'aperture_sum_29x60', 'aperture_sum_30x60', 'xcenter', 'ycenter'],
      dtype='object', length=402)

In [10]:
if not hasattr(planet, 'photometry_df'):
    # planet.load_dict(joblib_filename)
    planet.clean_cosmic_rays()
    planet.calibration_trace_location()
    planet.identify_trace_direction()
    planet.simple_phots()
    planet.center_all_traces()
    planet.fit_trace_slopes()
    planet.compute_sky_background(subpixels=32)
    planet.compute_columnwise_sky_background()

# Run Multi-Phot

In [11]:
if not hasattr(planet, 'photometry_df'):
    # Set up the list of aperture widths and heights to search
    min_aper_width = 1
    max_aper_width = 100
    min_aper_height = 1
    max_aper_height = 100

    aper_widths = np.arange(min_aper_width, max_aper_width + 2, 5)
    aper_heights = np.arange(min_aper_height, max_aper_height + 2, 5)
    
    planet.do_multi_phot(aper_widths, aper_heights)

# Determine the 'best' photometry SNR

In [12]:
med_photometry_df = np.median(planet.photometry_df, axis=0)
planet.normed_photometry_df = planet.photometry_df / med_photometry_df
planet.normed_uncertainty_df = np.sqrt(planet.photometry_df) / med_photometry_df

In [13]:
planet_fine_photometry_df = planet.photometry_df.copy()

In [14]:
fine_snr_lightcurves = create_raw_lc_stddev(planet)
fine_min_snr = fine_snr_lightcurves[fine_snr_lightcurves.argmin()]
fine_min_snr_colname = planet.photometry_df.columns[fine_snr_lightcurves.argmin()]
fine_min_snr_flux = planet.normed_photometry_df[fine_min_snr_colname]
fine_min_snr_uncs = planet.normed_uncertainty_df[fine_min_snr_colname]
fine_temp = fine_min_snr_colname.split('_')[-1].split('x')
fine_min_snr_aper_width, fine_min_snr_aper_height = np.int32(fine_temp)

In [15]:
info_message(f'Fine Aperture Photometry Resulted in {fine_min_snr:0.0f}ppm with '
             f'{fine_min_snr_aper_width}x{fine_min_snr_aper_height} aperture size; '
             f'with median uncertainties of {np.median(fine_min_snr_uncs)*1e6:0.0f} ppm')

[INFO] Fine Aperture Photometry Resulted in 209ppm with 20x50 aperture size; with median uncertainties of 126 ppm


# Configure system for PyMC3

In [16]:
idx_fwd = planet.idx_fwd
idx_rev = planet.idx_rev

In [17]:
# Compute a limb-darkened light curve using starry
times = planet.times
u = []
flux = planet.normed_photometry_df[fine_min_snr_colname]
yerr = planet.normed_uncertainty_df[fine_min_snr_colname]

But the real power comes from the fact that this is defined as a [Theano operation](http://deeplearning.net/software/theano/extending/extending_theano.html) so it can be combined with PyMC3 to do transit inference using Hamiltonian Monte Carlo.

## The transit model in PyMC3

In this section, we will construct a simple transit fit model using *PyMC3* and then we will fit a two planet model to simulated data.
To start, let's randomly sample some periods and phases and then define the time sampling:

Then, define the parameters.
In this simple model, we'll just fit for the limb darkening parameters of the star, and the period, phase, impact parameter, and radius ratio of the planets (note: this is already 10 parameters and running MCMC to convergence using [emcee](https://emcee.readthedocs.io) would probably take at least an hour).
For the limb darkening, we'll use a quadratic law as parameterized by [Kipping (2013)](https://arxiv.org/abs/1308.0009).
This reparameterizations is implemented in *exoplanet* as custom *PyMC3* distribution :class:`exoplanet.distributions.QuadLimbDark`.

In [18]:
print(f'This instance has {mp.cpu_count()} CPUs')

This instance has 16 CPUs


In [19]:
wasp43 = exoMAST_API('WASP43b')
t0_wasp43 = wasp43.transit_time  # 55528.3684  # exo.mast.stsci.edu
period_wasp43 = wasp43.orbital_period
n_epochs = np.int(np.round(((np.median(times) - t0_wasp43) / period_wasp43)-0.5))
t0_guess = t0_wasp43 + (n_epochs+0.5) * period_wasp43

In [20]:
b_wasp43 = wasp43.impact_parameter # 0.66 # Hellier 2011
u = [0]
edepth = np.sqrt(500/1e6)

orbit_wasp43 = xo.orbits.KeplerianOrbit(period=period_wasp43, t0=t0_guess, b=b_wasp43)
injected_light_curves = xo.LimbDarkLightCurve(u).get_light_curve(orbit=orbit_wasp43, r=edepth, t=times).eval().flatten()

# Run 400 MCMCs

In [21]:
fine_snr_flux = planet.normed_photometry_df
fine_snr_uncs = planet.normed_uncertainty_df

n_columns = len(fine_snr_flux.columns)

aper_sum_columns = planet.normed_photometry_df.drop(
    ['xcenter', 'ycenter'], axis=1).columns

xcenters = planet.photometry_df['xcenter']
ycenters = planet.photometry_df['ycenter']
xcenters_mod = xcenters - np.median(xcenters)
ycenters_mod = ycenters - np.median(ycenters)
times_mod = times - t0_guess#np.median(times)

In [35]:
n_space = 2
near_best_apertures_NxN_small = [f'aperture_sum_{aper_width_}x{aper_height_}' 
                       for aper_width_ in (np.arange(-n_space+1,n_space)+fine_min_snr_aper_width)
                       for aper_height_ in (np.arange(-n_space+1,n_space)+fine_min_snr_aper_height)]

In [36]:
near_best_apertures_NxN_small,len(near_best_apertures_NxN_small)

(['aperture_sum_18x48',
  'aperture_sum_18x49',
  'aperture_sum_18x50',
  'aperture_sum_18x51',
  'aperture_sum_18x52',
  'aperture_sum_19x48',
  'aperture_sum_19x49',
  'aperture_sum_19x50',
  'aperture_sum_19x51',
  'aperture_sum_19x52',
  'aperture_sum_20x48',
  'aperture_sum_20x49',
  'aperture_sum_20x50',
  'aperture_sum_20x51',
  'aperture_sum_20x52',
  'aperture_sum_21x48',
  'aperture_sum_21x49',
  'aperture_sum_21x50',
  'aperture_sum_21x51',
  'aperture_sum_21x52',
  'aperture_sum_22x48',
  'aperture_sum_22x49',
  'aperture_sum_22x50',
  'aperture_sum_22x51',
  'aperture_sum_22x52'],
 25)

# RUN ALL 12 IN ONE FUNCTION

In [ ]:
from arctor.utils import run_all_12_options
plt.rcParams['figure.figsize'] = (10, 6)

tune = 3000
draws = 3000
target_accept = 0.9
do_mcmc = False
save_as_you_go = False

base_name = 'WASP43_fine_grain_photometry_208ppm'
base_name = f'{base_name}_near_best_{n_space}x{n_space}'
near_best_apertures_NxN_small = [fine_min_snr_colname]  # aper_sum_columns,  #

mcmc_fits_all_12 = run_all_12_options(times,
                                      fine_snr_flux,
                                      fine_snr_uncs,
                                      near_best_apertures_NxN_small,
                                      xcenters=planet.trace_xcenters - np.median(planet.trace_xcenters),
                                      ycenters=planet.trace_ycenters - np.median(planet.trace_ycenters),
                                      trace_angles=planet.trace_angles - np.median(planet.trace_angles),
                                      trace_lengths=planet.trace_lengths - np.median(planet.trace_lengths),
                                      t0=t0_guess,
                                      u=u,
                                      period=period_wasp43,
                                      b=b_wasp43,
                                      idx_fwd=idx_fwd,
                                      idx_rev=idx_rev,
                                      tune=tune,
                                      draws=draws,
                                      target_accept=target_accept,
                                      do_mcmc=do_mcmc,
                                      save_as_you_go=save_as_you_go,
                                      injected_light_curve=1.0,
                                      working_dir=working_dir,
                                      base_name=base_name)

# RUN ALL 12 MANUALLY

In [ ]:
start0 = time()
# Linear Eclipse Depths with Negative Allowed
start1 = time()
print('Linear Eclipse depth fits - Allow Negative Eclipse Depths')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=None,
    allow_negative_edepths=True,
    use_rev_fwd_split=False,
    use_log_edepth=False
)

fine_grain_mcmcs_no_xcenter_lin_edepth_no_split_w_negEcl = fine_grain_mcmcs
filename_no_xcenter_lin_edepth_no_split_w_negEcl = filename
print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - Allow Negative Eclipse Depths with splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=None,  # SAME
    allow_negative_edepths=True,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=False  # SAME
)
fine_grain_mcmcs_with_no_xcenter_lin_edepth_w_split_w_negEcl = fine_grain_mcmcs
filename_with_no_xcenter_lin_edepth_w_split_w_negEcl = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - Allow Negative Eclipse Depths with xcenter')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # DIFFERENT
    allow_negative_edepths=True,  # SAME
    use_rev_fwd_split=False,  # SAME
    use_log_edepth=False,  # SAME
)

fine_grain_mcmcs_with_w_xcenter_lin_edepth_no_split_w_negEcl = fine_grain_mcmcs
filename_with_w_xcenter_lin_edepth_no_split_w_negEcl = filename
print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - '
      'Allow Negative Eclipse Depths with xcenter and splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # SAME
    allow_negative_edepths=True,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=False  # SAME
)

fine_grain_mcmcs_with_w_xcenter_lin_edepth_w_split_w_negEcl = fine_grain_mcmcs
filename_with_w_xcenter_lin_edepth_w_split_w_negEcl = filename
print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
# Linear Eclipse Depths without Negative Allowed
print('Linear Eclipse depth fits - Default everything')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43, idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    xcenters=None,  # DIFFERENT
    allow_negative_edepths=False,  # DIFFERENT
    use_rev_fwd_split=False,  # DIFFERENT
    use_log_edepth=False  # SAME
)

fine_grain_mcmcs_with_no_xcenter_lin_edepth_no_split = fine_grain_mcmcs
filename_with_no_xcenter_lin_edepth_no_split = filename
print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - Everything with splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43, idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    xcenters=None,  # SAME
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=False,  # SAME
)

fine_grain_mcmcs_with_no_xcenter_lin_edepth_w_split = fine_grain_mcmcs
filename_with_no_xcenter_lin_edepth_w_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - Everything with xcenter')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # DIFFERENT
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=False,  # DIFFERENT
    use_log_edepth=False  # SAME
)

fine_grain_mcmcs_with_w_xcenter_lin_edepth_no_split = fine_grain_mcmcs
filename_with_w_xcenter_lin_edepth_no_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Linear Eclipse depth fits - '
      'Everything with xcenter and splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # SAME
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=False)  # SAME

fine_grain_mcmcs_with_w_xcenter_lin_edepth_w_split = fine_grain_mcmcs
filename_with_w_xcenter_lin_edepth_w_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

# Logarithmic Eclipse Depths
start1 = time()
print('Log Eclipse depth fits - Default everything')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=None,  # DIFFERENT
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=False,  # DIFFERENT
    use_log_edepth=True  # DIFFERENT
)

fine_grain_mcmcs_with_no_xcenter_log_edepth_no_split = fine_grain_mcmcs
filename_with_no_xcenter_log_edepth_no_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Log Eclipse depth fits - Everything with splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=None,  # SAME
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=True  # SAME
)

fine_grain_mcmcs_with_no_xcenter_log_edepth_w_split = fine_grain_mcmcs
filename_with_no_xcenter_log_edepth_w_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

start1 = time()
print('Log Eclipse depth fits - Everything with xcenter')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # DIFFERENT
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=False,  # DIFFERENT
    use_log_edepth=True  # SAME
)

fine_grain_mcmcs_with_w_xcenter_log_edepth_no_split = fine_grain_mcmcs
filename_with_w_xcenter_log_edepth_no_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

del fine_grain_mcmcs, filename

start1 = time()
print('Log Eclipse depth fits - Everything with xcenter and splitting fwd rev')
fine_grain_mcmcs, filename = run_multiple_pymc3(
    times, fine_snr_flux, fine_snr_uncs, near_best_apertures_NxN_small,
    t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
    idx_fwd=idx_fwd, idx_rev=idx_rev,
    tune=tune, draws=draws, target_accept=target_accept,
    do_mcmc=do_mcmc, save_as_you_go=save_as_you_go,
    injected_light_curve=1.0, base_name=base_name, working_dir=working_dir,
    xcenters=xcenters_mod,  # SAME
    allow_negative_edepths=False,  # SAME
    use_rev_fwd_split=True,  # DIFFERENT
    use_log_edepth=True  # SAME
)

fine_grain_mcmcs_with_w_xcenter_log_edepth_w_split = fine_grain_mcmcs
filename_with_w_xcenter_log_edepth_w_split = filename

print(f'[INFO] This MCMCs took {(time() - start1)/60:0.2f} minutes')

print(f'[INFO] All 12 MCMCs took {(time() - start0)/60:0.2f} minutes')

# Combine all 12 manually

In [ ]:
mcmc_fits = [fine_grain_mcmcs_with_no_xcenter_lin_edepth_no_split_w_negEcl,
             fine_grain_mcmcs_with_no_xcenter_lin_edepth_w_split_w_negEcl,
             fine_grain_mcmcs_with_w_xcenter_lin_edepth_no_split_w_negEcl,
             fine_grain_mcmcs_with_w_xcenter_lin_edepth_w_split_w_negEcl,
             fine_grain_mcmcs_with_no_xcenter_lin_edepth_no_split,
             fine_grain_mcmcs_with_no_xcenter_lin_edepth_w_split,
             fine_grain_mcmcs_with_w_xcenter_lin_edepth_no_split,
             fine_grain_mcmcs_with_w_xcenter_lin_edepth_w_split,
             fine_grain_mcmcs_with_no_xcenter_log_edepth_no_split,
             fine_grain_mcmcs_with_no_xcenter_log_edepth_w_split,
             fine_grain_mcmcs_with_w_xcenter_log_edepth_no_split,
             fine_grain_mcmcs_with_w_xcenter_log_edepth_w_split
            ]

In [ ]:
flux = planet.normed_photometry_df[fine_min_snr_colname]
trace_angles = planet.trace_angles
fit_2D_time_vs_other(times, flux, trace_angles, idx_fwd, idx_rev, 
                     label='trace_angles,time vs flux', n_sig=3)

In [ ]:
flux = planet.normed_photometry_df[fine_min_snr_colname]
trace_lengths = planet.trace_lengths
fit_2D_time_vs_other(times, flux, trace_lengths, idx_fwd, idx_rev, 
                     label='trace_angles,time vs flux', n_sig=3)

In [ ]:
flux = planet.normed_photometry_df[fine_min_snr_colname]
trace_angles = planet.trace_angles
fit_2D_time_vs_other(times, flux, xcenters, idx_fwd, idx_rev, 
                     label='trace_angles,time vs flux', n_sig=3)

In [ ]:
flux = planet.normed_photometry_df[fine_min_snr_colname]
trace_angles = planet.trace_angles
fit_2D_time_vs_other(times, flux, ycenters, idx_fwd, idx_rev, 
                     label='trace_angles,time vs flux', n_sig=3)

# Testing new "all correlations and all flavors" MCMC

In [27]:
def compute_sdnr(map_soln, phots, idx_fwd, idx_rev):
    ppm = 1e6
    phots_std_fwd = phots[idx_fwd].std()
    phots_std_rev = phots[idx_rev].std()
    phots_std = np.mean([phots_std_fwd, phots_std_rev])
    
    if 'mean_fwd' not in map_soln.keys():
        map_model = map_soln['light_curves'].flatten() + map_soln['line_model']
    else:
        map_model = np.zeros_like(times)
        map_model[idx_fwd] = map_soln['light_curves_fwd'].flatten() + map_soln['line_model_fwd']
        map_model[idx_rev] = map_soln['light_curves_rev'].flatten() + map_soln['line_model_rev']
    
    varnames = [key for key in map_soln.keys() if '__' not in key and 'light' not in key and 'line' not in key and 'le_edepth_0' not in key]
    
    res_fwd = np.std(map_model[idx_fwd] - phots[idx_fwd])
    res_rev = np.std(map_model[idx_rev] - phots[idx_rev])
    res_std = np.mean([res_fwd, res_rev])
    
    print(f'{str(varnames):<80}')
    print(f'{res_std*ppm:0.2f}, {phots_std*ppm:0.2f}, {(phots_std - res_std)*ppm:0.2f} ppm difference'),
    
    return res_std*ppm, phots_std*ppm, (phots_std - res_std)*ppm

# HERE

In [109]:
n_space = 10
near_best_apertures_NxN_small = [f'aperture_sum_{aper_width_}x{aper_height_}' 
                                 for aper_width_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_width)
                                 for aper_height_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_height)]
len(near_best_apertures_NxN_small)#, near_best_apertures_NxN_small

100

In [96]:
from arctor.utils import run_pymc3_both

tune = 3000
draws = 3000
target_accept = 0.9
do_mcmc = True
idx_fwd_ = None
idx_rev_ = None
use_log_edepth = False
allow_negative_edepths = False

xcenters_mod = planet.trace_xcenters - np.median(planet.trace_xcenters)
ycenters_mod = planet.trace_ycenters - np.median(planet.trace_ycenters)
trace_angles_mod = planet.trace_angles - np.median(planet.trace_angles)
trace_lengths_mod = planet.trace_lengths - np.median(planet.trace_lengths)
phots = planet.normed_photometry_df[fine_min_snr_colname].values

# n_space = 10
# near_best_apertures_NxN_small = [f'aperture_sum_{aper_width_}x{aper_height_}' 
#                                  for aper_width_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_width)
#                                  for aper_height_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_height)]
# # len(near_best_apertures_NxN_small)# near_best_apertures_NxN_small

decor_span_MAPs_only_list = []
startn1 = time()
for aper_column in near_best_apertures_NxN_small:  # planet.normed_photometry_df.columns:  # [fine_min_snr_colname]:
    phots = planet.normed_photometry_df[aper_column].values
    uncs = planet.normed_uncertainty_df[aper_column].values
    
    start0 = time()
    for idx_fwd_, idx_rev_ in zip([None, idx_fwd], [None, idx_rev]):
        start1 = time()
        for xcenters_ in [None, xcenters_mod]:
            start2 = time()
            for ycenters_ in [None, ycenters_mod]:
                start3 = time()
                for trace_angles_ in [None, trace_angles_mod]:
                    start4 = time()
                    for trace_lengths_ in [None, trace_lengths_mod]:
                        start5 = time()
                        fine_grain_mcmcs, map_soln = run_pymc3_both(
                            times, phots, uncs,
                            t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
                            tune=tune, draws=draws, target_accept=target_accept,
                            xcenters=xcenters_,
                            ycenters=ycenters_,
                            trace_angles=trace_angles_,
                            trace_lengths=trace_lengths_,
                            do_mcmc=False,
                            idx_fwd=idx_fwd_,
                            idx_rev=idx_rev_,
                            use_log_edepth=use_log_edepth,
                            allow_negative_edepths=allow_negative_edepths)

                        info_message(f'Full decorrelation took {(time() - start5)/60:0.2f} minutes')
                        res_std_ppm, phots_std_ppm, res_diff_ppm = compute_sdnr(map_soln, phots, idx_fwd, idx_rev)
                        save_state = [idx_fwd_ is None, idx_rev_ is None, xcenters_ is None, ycenters_ is None, trace_angles_ is None, trace_lengths_ is None]
                        save_state.extend([aper_column, fine_grain_mcmcs, map_soln])
                        save_state.extend([res_std_ppm, phots_std_ppm, res_diff_ppm])
                        decor_span_MAPs_only_list.append(save_state)
                    info_message(f'Trace Length Option took {(time() - start4)/60:0.2f} minutes')
                info_message(f'Trace Angle Option took {(time() - start3)/60:0.2f} minutes')
            info_message(f'Ycenter Option took {(time() - start2)/60:0.2f} minutes')
        info_message(f'xcenter Option took {(time() - start1)/60:0.2f} minutes')
    info_message(f'Run through for {aper_column} took {(time() - start0)/60:0.2f} minutes')
info_message(f'Entire loop-d-loop took {(time() - startn1)/60:0.2f} minutes')

joblib.dump(decor_span_MAPs_only_list, 'results_decor_span_MAPs_only_SDNR.joblib.save')

optimizing logp for variables: [slope, edepth, mean]
97it [00:00, 170.11it/s, logp=4.343307e+02]
message: Desired error not necessarily achieved due to precision loss.
logp: -14782.210393473411 -> 434.3306535995832


[INFO] Map Soln Edepth:8.301140543661763
[INFO] Full decorrelation took 0.03 minutes
['mean', 'edepth', 'slope']                                                     
202.65, 206.87, 4.22 ppm difference


optimizing logp for variables: [slope_trace_length, slope, edepth, mean]
116it [00:00, 188.48it/s, logp=4.862688e+02]
message: Desired error not necessarily achieved due to precision loss.
logp: -14783.59668783453 -> 486.26877111318686


[INFO] Map Soln Edepth:10.709987372562617
[INFO] Full decorrelation took 0.04 minutes
['mean', 'edepth', 'slope', 'slope_trace_length']                               
204.25, 206.87, 2.62 ppm difference
[INFO] Trace Length Option took 0.08 minutes


optimizing logp for variables: [slope_trace_angle, slope, edepth, mean]
144it [00:00, 210.47it/s, logp=4.355306e+02]
message: Desired error not necessarily achieved due to precision loss.
logp: -14783.59668783453 -> 435.5305763499394


...

optimizing logp for variables: [slope_trace_angle, slope_ycenter, slope_xcenter, slope, edepth, mean_rev, mean_fwd]
53it [00:00, 83.93it/s, logp=5.178430e+02] 
message: Optimization terminated successfully.
logp: -14815.872776060343 -> 517.8429620230279


[INFO] Map Soln Edepth:13.186043147240529
[INFO] Full decorrelation took 0.17 minutes
['mean_fwd', 'mean_rev', 'edepth', 'slope', 'slope_xcenter', 'slope_ycenter', 'slope_trace_angle']
173.68, 206.79, 33.11 ppm difference


optimizing logp for variables: [slope_trace_length, slope_trace_angle, slope_ycenter, slope_xcenter, slope, edepth, mean_rev, mean_fwd]
115it [00:00, 140.47it/s, logp=5.190711e+02]
message: Desired error not necessarily achieved due to precision loss.
logp: -14817.259070421464 -> 519.0710678179572


[INFO] Map Soln Edepth:14.600556609200803
[INFO] Full decorrelation took 0.08 minutes
['mean_fwd', 'mean_rev', 'edepth', 'slope', 'slope_xcenter', 'slope_ycenter', 'slope_trace_angle', 'slope_trace_length']
170.28, 206.79, 36.51 ppm difference
[INFO] Trace Length Option took 0.25 minutes
[INFO] Trace Angle Option took 0.53 minutes
[INFO] Ycenter Option took 1.16 minutes
[INFO] xcenter Option took 2.39 minutes
[INFO] Run through for aperture_sum_25x55 took 3.67 minutes
[INFO] Entire loop-d-loop took 275.37 minutes


['results_decor_span_MAPs_only_SDNR.joblib.save']

In [106]:
n_space = 5
near_best_apertures_NxN_small = [f'aperture_sum_{aper_width_}x{aper_height_}' 
                                 for aper_width_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_width)
                                 for aper_height_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_height)]
len(near_best_apertures_NxN_small), near_best_apertures_NxN_small

(25,
 ['aperture_sum_18x48',
  'aperture_sum_18x49',
  'aperture_sum_18x50',
  'aperture_sum_18x51',
  'aperture_sum_18x52',
  'aperture_sum_19x48',
  'aperture_sum_19x49',
  'aperture_sum_19x50',
  'aperture_sum_19x51',
  'aperture_sum_19x52',
  'aperture_sum_20x48',
  'aperture_sum_20x49',
  'aperture_sum_20x50',
  'aperture_sum_20x51',
  'aperture_sum_20x52',
  'aperture_sum_21x48',
  'aperture_sum_21x49',
  'aperture_sum_21x50',
  'aperture_sum_21x51',
  'aperture_sum_21x52',
  'aperture_sum_22x48',
  'aperture_sum_22x49',
  'aperture_sum_22x50',
  'aperture_sum_22x51',
  'aperture_sum_22x52'])

In [110]:
decor_aper_columns_list = []
for aper_column in near_best_apertures_NxN_small:  # planet.normed_photometry_df.columns:  # [fine_min_snr_colname]:
    for idx_fwd_, idx_rev_ in zip([None, idx_fwd], [None, idx_rev]):
        for xcenters_ in [None, xcenters_mod]:
            for ycenters_ in [None, ycenters_mod]:
                for trace_angles_ in [None, trace_angles_mod]:
                    for trace_lengths_ in [None, trace_lengths_mod]:
                        decor_aper_columns_list.append(aper_column)

joblib.dump(decor_aper_columns_list, 'decor_span_MAPs_only_aper_columns_list.joblib.save')

['decor_span_MAPs_only_aper_columns_list.joblib.save']

In [111]:
len(decor_aper_columns_list)

3200

In [98]:
from arctor.utils import run_pymc3_both

tune = 3000
draws = 3000
target_accept = 0.9
do_mcmc = True
idx_fwd_ = None
idx_rev_ = None
use_log_edepth = False
allow_negative_edepths = False

xcenters_mod = planet.trace_xcenters - np.median(planet.trace_xcenters)
ycenters_mod = planet.trace_ycenters - np.median(planet.trace_ycenters)
trace_angles_mod = planet.trace_angles - np.median(planet.trace_angles)
trace_lengths_mod = planet.trace_lengths - np.median(planet.trace_lengths)
phots = planet.normed_photometry_df[fine_min_snr_colname].values

# near_best_apertures_NxN_small = [fine_min_snr_colname]
n_space = 5
near_best_apertures_NxN_small = [f'aperture_sum_{aper_width_}x{aper_height_}' 
                                 for aper_width_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_width)
                                 for aper_height_ in (np.arange(-n_space//2+1, n_space//2+1)+fine_min_snr_aper_height)]

decor_span_MCMCs_list = []
startn1 = time()
for aper_column in near_best_apertures_NxN_small:  # [fine_min_snr_colname]:  # planet.normed_photometry_df.columns:  # near_best_apertures_NxN_small:  # 
    phots = planet.normed_photometry_df[aper_column].values
    uncs = planet.normed_uncertainty_df[aper_column].values
    
    start0 = time()
    for idx_fwd_, idx_rev_ in zip([None, idx_fwd], [None, idx_rev]):
        start1 = time()
        for xcenters_ in [None, xcenters_mod]:
            start2 = time()
            for ycenters_ in [None, ycenters_mod]:
                start3 = time()
                for trace_angles_ in [None, trace_angles_mod]:
                    start4 = time()
                    for trace_lengths_ in [None, trace_lengths_mod]:
                        start5 = time()
                        fine_grain_mcmcs, map_soln = run_pymc3_both(
                            times, phots, uncs,
                            t0=t0_guess, u=u, period=period_wasp43, b=b_wasp43,
                            tune=tune, draws=draws, target_accept=target_accept,
                            xcenters=xcenters_,
                            ycenters=ycenters_,
                            trace_angles=trace_angles_,
                            trace_lengths=trace_lengths_,
                            do_mcmc=do_mcmc,
                            idx_fwd=idx_fwd_,
                            idx_rev=idx_rev_,
                            use_log_edepth=use_log_edepth,
                            allow_negative_edepths=allow_negative_edepths)
                        
                        info_message(f'Full decorrelation took {(time() - start5)/60:0.2f} minutes')
                        res_std_ppm, phots_std_ppm, res_diff_ppm = compute_sdnr(map_soln, phots, idx_fwd, idx_rev)
                        save_state = [idx_fwd_ is None, idx_rev is None, xcenters_ is None, ycenters_ is None, trace_angles_ is None, trace_lengths_ is None]
                        save_state.extend([aper_column, fine_grain_mcmcs, map_soln])
                        save_state.extend([res_std_ppm, phots_std_ppm, res_diff_ppm])
                        decor_span_MCMCs_list.append(save_state)
                    info_message(f'Trace Length Option took {(time() - start4)/60:0.2f} minutes')
                info_message(f'Trace Angle Option took {(time() - start3)/60:0.2f} minutes')
            info_message(f'Ycenter Option took {(time() - start2)/60:0.2f} minutes')
        info_message(f'xcenter Option took {(time() - start1)/60:0.2f} minutes')
    info_message(f'Run through for {aper_column} took {(time() - start0)/60:0.2f} minutes')
info_message(f'Entire loop-d-loop took {(time() - startn1)/60:0.2f} minutes')

joblib.dump(decor_span_MCMCs_list, 'results_decor_span_MCMCs_25_bestest_SDNR.joblib.save')

optimizing logp for variables: [slope, edepth, mean]
165it [00:00, 245.46it/s, logp=4.338431e+02]
message: Desired error not necessarily achieved due to precision loss.
logp: -14799.957117518825 -> 433.84312344295927


[INFO] Map Soln Edepth:8.281821778829165


Multiprocess sampling (16 chains in 16 jobs)
Multiprocess sampling (16 chains in 16 jobs)
NUTS: [slope, edepth, mean]
NUTS: [slope, edepth, mean]
Sampling 16 chains: 100%|██████████| 96000/96000 [00:46<00:00, 2053.51draws/s]
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


[INFO] Full decorrelation took 1.00 minutes
['mean', 'edepth', 'slope']                                                     
202.19, 206.36, 4.17 ppm difference


optimizing logp for variables: [slope_trace_length, slope, edepth, mean]
31it [00:00, 79.84it/s, logp=4.864094e+02] 
message: Optimization terminated successfully.
logp: -14801.343411879943 -> 486.4093702034808


...

[INFO] Full decorrelation took 1.61 minutes
['mean', 'edepth', 'slope', 'slope_xcenter', 'slope_ycenter']                   
172.69, 206.10, 33.41 ppm difference


optimizing logp for variables: [slope_trace_length, slope_ycenter, slope_xcenter, slope, edepth, mean]
36it [00:00, 82.45it/s, logp=5.227114e+02] 
message: Optimization terminated successfully.
logp: -14824.990276347185 -> 522.711390913816


[INFO] Map Soln Edepth:14.583860618089698


Multiprocess sampling (16 chains in 16 jobs)
Multiprocess sampling (16 chains in 16 jobs)
NUTS: [slope_trace_length, slope_ycenter, slope_xcenter, slope, edepth, mean]
NUTS: [slope_trace_length, slope_ycenter, slope_xcenter, slope, edepth, mean]


OSError: [Errno 12] Cannot allocate memory

In [ ]:
mcmc_fit_ = fine_grain_mcmcs_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl
map_soln_ = map_soln_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl
# map_soln_ = map_soln_no_decor_lin_edepth_no_split_w_negEcl
# mcmc_fits_ = fine_grain_mcmcs_no_decor_lin_edepth_no_split_w_negEcl
ppm = 1e6

phots = planet.normed_photometry_df[fine_min_snr_colname].values
phots_std = phots.std()*ppm
varnames = [key for key in map_soln_.keys() if '__' not in key and 'light' not in key and 'line' not in key and 'le_edepth_0' not in key]
label_mcmc_fit_ = str(varnames)

if 'mean_fwd' not in map_soln_.keys():
    map_model = map_soln_['light_curves'].flatten() + map_soln_['mean'] + map_soln_['slope']*(times - np.median(times))
    if 'slope_xcenter' in map_soln_.keys():
        map_model = map_model + map_soln_['slope_xcenter']*(xcenters.values - np.median(xcenters))
else:
    map_model = np.zeros_like(times)
    map_model[idx_fwd] = map_soln_['light_curves_fwd'].flatten() + map_soln_['mean_fwd'] + map_soln_['slope']*(times[idx_fwd] - np.median(times))
    map_model[idx_rev] = map_soln_['light_curves_rev'].flatten() + map_soln_['mean_rev'] + map_soln_['slope']*(times[idx_rev] - np.median(times))

    if 'slope_xcenter' in map_soln_.keys():
        map_model = map_model + map_soln_['slope_xcenter']*(xcenters.values - np.median(xcenters))

print(f'{label_mcmc_fit_}: {phots_std - (np.std(map_model - phots))*ppm:0.2f} ppm difference')
print(pm.summary(mcmc_fit_, varnames=varnames)[['mean', 'sd', 'hpd_2.5', 'hpd_97.5']])
setup_and_plot_GTC({'trace':mcmc_fit_, 'map_soln':map_soln_},
                   smoothingKernel=0.1, square_edepth=False,
                   varnames=varnames)

In [ ]:
# mcmc_fit_ = fine_grain_mcmcs_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl
# map_soln_ = map_soln_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl
map_soln_ = map_soln_no_decor_lin_edepth_no_split_w_negEcl
mcmc_fits_ = fine_grain_mcmcs_no_decor_lin_edepth_no_split_w_negEcl
ppm = 1e6

phots = planet.normed_photometry_df[fine_min_snr_colname].values
phots_std = phots.std()*ppm
varnames = [key for key in map_soln_.keys() if '__' not in key and 'light' not in key and 'line' not in key and 'le_edepth_0' not in key]
label_mcmc_fit_ = str(varnames)

if 'mean_fwd' not in map_soln_.keys():
    map_model = map_soln_['light_curves'].flatten() + map_soln_['mean'] + map_soln_['slope']*(times - np.median(times))
    if 'slope_xcenter' in map_soln_.keys():
        map_model = map_model + map_soln_['slope_xcenter']*(xcenters.values - np.median(xcenters))
else:
    map_model = np.zeros_like(times)
    map_model[idx_fwd] = map_soln_['light_curves_fwd'].flatten() + map_soln_['mean_fwd'] + map_soln_['slope']*(times[idx_fwd] - np.median(times))
    map_model[idx_rev] = map_soln_['light_curves_rev'].flatten() + map_soln_['mean_rev'] + map_soln_['slope']*(times[idx_rev] - np.median(times))
    map_model[idx_rev] = map_model[idx_rev] + 
    # if 'slope_xcenter' in map_soln_.keys():
    #     map_model = map_model + map_soln_['slope_xcenter']*(xcenters.values - np.median(xcenters))

print(f'{label_mcmc_fit_}: {phots_std - (np.std(map_model - phots))*ppm:0.2f} ppm difference')
print(pm.summary(mcmc_fit_, varnames=varnames)[['mean', 'sd', 'hpd_2.5', 'hpd_97.5']])
setup_and_plot_GTC({'trace':mcmc_fit_, 'map_soln':map_soln_},
                   smoothingKernel=0.1, square_edepth=False,
                   varnames=varnames)

In [ ]:
start = time()
varnames_ = varnames
pm.traceplot(mcmc_fit_, var_names=varnames_);
info_message(f'Time to plot {len(varnames_)} vars: {time() - start:0.2f} seconds')

# Run  through All 12 (original) MCMC Flavors to Compare results

In [ ]:
mcmc_fits = [{fine_min_snr_colname:{'trace':fine_grain_mcmcs_no_decor_lin_edepth_no_split_w_negEcl,
             'map_soln':map_soln_no_decor_lin_edepth_no_split_w_negEcl}},
            {fine_min_snr_colname:{'trace':fine_grain_mcmcs_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl,
             'map_soln':map_soln_w_xcenter_w_ycenter_w_trace_angle_w_trace_length_lin_edepth_no_split_w_negEcl}}]

In [ ]:
label_mcmc_fit = []
for mcmc_fit_ in mcmc_fits:
    label_mcmc_fit.append(str([key for key in mcmc_fit_[fine_min_snr_colname]['map_soln'].keys() if '__' not in key and 'light' not in key]))

for k in range(len(mcmc_fits)):
    label_mcmc_fit[k] = label_mcmc_fit[k][:-1] + ", 'allow_neg_slopes']"

In [ ]:
ppm = 1e6
phots = planet.normed_photometry_df[fine_min_snr_colname].values
phots_std = phots.std()
for k, mcmc_fit_ in enumerate(mcmc_fits):
    map_soln = mcmc_fit_[fine_min_snr_colname]['map_soln']
    if 'mean_fwd' not in map_soln.keys():
        map_model = map_soln['light_curves'].flatten() + map_soln['line_model']
    else:
        map_model = np.zeros_like(times)
        map_model[idx_fwd] = map_soln['light_curves_fwd'].flatten() + map_soln['line_model_fwd']
        map_model[idx_rev] = map_soln['light_curves_rev'].flatten() + map_soln['line_model_rev']

    print(f'{label_mcmc_fit[k]:<80}')
    print(f'{phots_std*ppm:0.2f}, {np.std(map_model - phots)*ppm:0.2f}, {phots_std*ppm - (np.std(map_model - phots))*ppm:0.2f} ppm difference')

In [ ]:
for k, mcmc_fit_ in enumerate(mcmc_fits):
    print(pm.summary(trace, varnames=varnames)[['mean', 'sd', 'hpd_2.5', 'hpd_97.5']])

In [ ]:
for k, mcmc_fit_ in enumerate(mcmc_fits):
    setup_and_plot_GTC(mcmc_fit_[fine_min_snr_colname])

# Defunct Analysis Steps

In [ ]:
[key for key in fine_grain_mcmcs_with_w_xcenter_lin_edepth_w_split['aperture_sum_20x50']['map_soln'].keys()
                if '__' not in key and 'light' not in key]

In [ ]:
pm.summary(trace, varnames=varnames)

In [ ]:
import pygtc
plt.rcParams['figure.figsize'] = 10,10

trace = fine_grain_mcmcs_w_xcenterfit[fine_min_snr_colname]['trace']

samples = pm.trace_to_dataframe(trace, varnames=varnames)
samples['edepth'] = 10**samples['log_edepth']

pygtc.plotGTC(samples, nContourLevels=3);

In [ ]:
from scipy.special import erf
sigmas = erf(np.arange(1,6)/np.sqrt(2))
print(np.percentile(samples['edepth'], sigmas*100)*1e6)
print(10**(np.percentile(samples['log_edepth'], sigmas*100))*1e6)

Now we can plot the simulated data and the maximum a posteriori model to make sure that our initialization looks ok.

## Sampling

Now, let's sample from the posterior defined by this model.
As usual, there are strong covariances between some of the parameters so we'll use :func:`exoplanet.get_dense_nuts_step`.

After sampling, it's important that we assess convergence.
We can do that using the `pymc3.summary` function:

In [ ]:
aper_sum_columns = planet.normed_photometry_df.drop(
    ['xcenter', 'ycenter'], axis=1).columns
aper_sum_columns

In [ ]:
trace_ = fine_grain_mcmcs_w_xcenterfit[fine_min_snr_colname]['trace']
dir(trace_)
trace_.report._gelman_rubin.values()
dir(trace_.report)
trace_.report._run_convergence_checks??

In [ ]:
# fine_grain_mcmcs_w_xcenterfit[fine_min_snr_colname]['map_soln']
edepths = []
means = []
slopes = []
slopes_xcenter = []

edepths_unc = []
means_unc = []
slopes_unc = []
slopes_xcenter_unc = []

mesh_widths = []
mesh_heights = []

for colname in tqdm(aper_sum_columns):
    aper_width_, aper_height_ = np.int32(colname.split('_')[-1].split('x'))
    mesh_widths.append(aper_width_)
    mesh_heights.append(aper_height_)

    # Load Summary from Colname
    summary_df = pm.summary(fine_grain_mcmcs_w_xcenterfit[colname]['trace'], varnames=varnames)

    # Store mean values
    edepths.append(summary_df['mean'].loc['edepth'])
    means.append(summary_df['mean'].loc['mean'])
    slopes.append(summary_df['mean'].loc['slope'])
    slopes_xcenter.append(summary_df['mean'].loc['slope_xcenter'])

    # Store uncertainties
    edepths_unc.append(summary_df['sd'].loc['edepth'])
    means_unc.append(summary_df['sd'].loc['mean'])
    slopes_unc.append(summary_df['sd'].loc['slope'])
    slopes_xcenter_unc.append(summary_df['sd'].loc['slope_xcenter'])

In [ ]:
df_columns = ["aper_width", "aper_height", "edepth", "mean", "slope", "slope_xcenter"]
means_df = pd.DataFrame(np.transpose([mesh_widths, mesh_heights, edepths, means, slopes, slopes_xcenter]), 
                        columns=df_columns)
uncs_df = pd.DataFrame(np.transpose([mesh_widths, mesh_heights, edepths_unc, means_unc, slopes_unc, slopes_xcenter_unc]), 
                        columns=df_columns)

In [ ]:
# mesh_widths_sorted = np.argsort(mesh_widths)
# mesh_heights_sorted = np.argsort(mesh_heights)
plt.plot(means_df['aper_width'], (means_df['edepth'] / uncs_df['edepth']), '.')
plt.plot(means_df['aper_height'], (means_df['edepth'] / uncs_df['edepth']),'.')

plt.figure()
plt.plot(means_df['aper_height']*means_df['aper_width'], (means_df['edepth'] / uncs_df['edepth']),'.')

In [ ]:
mesh_widths_sorted = np.argsort(mesh_widths)
mesh_heights_sorted = np.argsort(mesh_heights)

plt.figure()
plt.plot(means_df['aper_width'], means_df['mean'],'.')
plt.plot(means_df['aper_height'], means_df['mean'],'.')

plt.figure()
plt.plot(means_df['aper_height']*means_df['aper_width'], means_df['mean'],'.')

That looks pretty good!
Fitting this without *exoplanet* would have taken a lot more patience.

Now we can also look at the [corner plot](https://corner.readthedocs.io) of some of that parameters of interest:

In [ ]:
import pygtc
import corner

In [ ]:
trace = fine_grain_mcmcs_w_xcenterfit[fine_min_snr_colname]['trace']

In [ ]:
n_lightcurves_fwd = trace['light_curves_fwd'].shape[0]
light_curves_fwd = trace['light_curves_fwd'].reshape(trace['light_curves_fwd'].shape[:2])#trace['mean_fwd'][:,None]

n_lightcurves_rev = trace['light_curves_rev'].shape[0]
light_curves_rev = trace['light_curves_rev'].reshape(trace['light_curves_rev'].shape[:2])#trace['mean_rev'][:,None]

for k in np.random.choice(np.arange(n_lightcurves_fwd), size=100):
    plt.plot(t[idx_fwd], light_curves_fwd[k], '.', color='C0', alpha=0.25)

for k in np.random.choice(np.arange(n_lightcurves_rev), size=100):
    plt.plot(t[idx_rev], light_curves_rev[k], '.', color='C1', alpha=0.25)

plt.ylim(-1e-5, 1e-6);
plt.xlim(t0_guess - 0.1, t0_guess + 0.1);

In [ ]:
for colname in tqdm(aper_sum_columns):
    trace = fine_grain_mcmcs_w_xcenterfit[colname]['trace']
    
    samples = pm.trace_to_dataframe(trace, varnames=varnames)
    truth = [0.0, 1.0, 1.0, 0.0]
    corner.corner(samples, truths=truth, labels=varnames);

## Phase plots

Like in the radial velocity tutorial (:ref:`rv`), we can make plots of the model predictions for each planet.

In [ ]:
line_fit.shape

In [ ]:
plt.figure()
# Get the posterior median orbital parameters
p = period
# t0 = np.median(trace["t0"])

# Plot the folded data
line_fit =  + trace['slope_xcenter'] + trace['slope']
line_fit = slope * (t-t0_guess) + mean + slope * (xcenters-np.median(xcenters))

plt.errorbar(times[idx_fwd] - t0, (data - line_fit + 1.0)[idx_fwd], yerr=yerr[idx_fwd], fmt=".", color='C0', label="fwd data", zorder=-1000)
plt.errorbar(times[idx_rev] - t0, (data - line_fit + 1.0)[idx_rev], yerr=yerr[idx_rev], fmt=".", color='C3', label="rev data", zorder=-1000)

# Plot the folded model
preds_fwd = trace["light_curves"][:,:,0]
# preds_rev = trace["light_curves"][:,:,0] + trace["mean"][:, None]
pred_fwd = np.median(preds_fwd, axis=0)
# pred_rev = np.median(preds_rev, axis=0)


plt.plot(times - t0, pred_fwd, color="C1", label="model", zorder=10)
# plt.plot(t[idx_rev] - t0, pred_rev, color="C2", label="model", zorder=10)
plt.axhline(1.0, ls='--', color='k')

# Annotate the plot with the planet's period
txt = f"Eclipse Depth = {np.mean(trace['edepth']*1e6):.0f}"
txt += f" +/- {np.std(trace['edepth']*1e6):.0f} ppm"

plt.annotate(
    txt,
    (0, 0),
    xycoords="axes fraction",
    xytext=(5, 5),
    textcoords="offset points",
    ha="left",
    va="bottom",
    fontsize=12,
)

add_traces = False
if add_traces:
    n_traces = 1000
    
    idx_rand = np.random.choice(np.arange(preds_fwd.shape[0]), size=n_traces, replace=False)
    for pred_ in preds_fwd[idx_rand]:
        plt.plot(times - t0, pred_, color="grey", alpha=0.5, zorder=0)

    # idx_rand = np.random.choice(np.arange(preds_fwd.shape[0]), size=n_traces, replace=False)
    # for pred_ in preds_rev[idx_rand]:
    #     plt.plot(t[idx_rev] - t0, pred_, color="grey", alpha=0.5, zorder=0)

plt.legend(fontsize=10, loc=4)
plt.xlim((times - t0).min(), (times - t0).max())
plt.xlabel("Time Since Eclipse [days]")
plt.ylabel("Relative Flux")
plt.title("WASP43 UVIS Eclipse");

## Citations

As described in the :ref:`citation` tutorial, we can use :func:`exoplanet.citations.get_citations_for_model` to construct an acknowledgement and BibTeX listing that includes the relevant citations for this model.
This is especially important here because we have used quite a few model components that should be cited.

In [ ]:
with model:
    txt, bib = xo.citations.get_citations_for_model()
print(txt)

In [ ]:
print("\n".join(bib.splitlines()[:10]) + "\n...")